# Notebook Goal: Raw Data Ingestion

## Purpose
Ingest raw data from source volumes into the Bronze layer of the Databricks Lakehouse.

## What This Notebook Does

- Reads multiple source datasets defined in a config file
- Writes data to the Bronze layer in Delta format
- Uses overwrite mode for table writes

## Output

Delta tables stored in the Bronze layer (`workspace.bronze`)

#  1. imports 

In [0]:
from config import INGESTION_CONFIG

# 2. Ingest Files into Bronze Tables

In [0]:
for item in INGESTION_CONFIG:
    try:
        print(f"Ingesting {item['source']} → workspace.bronze.{item['table']}")

        df = (
            spark.read
                 .option("header", "true")
                 .option("inferSchema", "true")
                 .csv(item["path"])
        )

        (
            df.write
              .mode("overwrite")
              .format("delta")
              .saveAsTable(f"workspace.bronze.{item['table']}")
        )

    except Exception as e:
        print(
            f"❌ Error ingesting {item.get('source', 'unknown source')} "
            f"into workspace.bronze.{item.get('table', 'unknown table')}: {e}"
        )
